In [246]:
import pandas as pd
import os
from skimage import io
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, classification_report, f1_score, roc_auc_score
import pickle
import Functions2
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report

In [229]:
# Load the data
data_path = r"/Users/bogdancristianmihaila/Desktop/2nd Semester/Github/project2/Project-2-Medical-Imaging/data/full_data.csv"
df = pd.read_csv(data_path)

# Preprocess the diagnostic column
df['diagnostic'] = df['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0})

# Define the function to extract features
def extract_features(folder_path):
    feature_1 = []
    feature_2 = []
    feature_3 = []
    feature_4 = []
    feature_5 = []
    feature_6 = []
    feature_7 = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            original = io.imread(image_path)

            # Ignore the alpha channel (e.g. transparency)
            if original.shape[-1] == 4:
                original = original[..., :3]

            feature_1.append(Functions2.measure_pigment_network(original))
            feature_2.append(Functions2.measure_blue_veil(original))
            feature_3.append(Functions2.measure_vascular(original))
            feature_4.append(Functions2.measure_globules(original))
            feature_5.append(Functions2.measure_streaks(original))
            feature_6.append(Functions2.measure_irregular_pigmentation(original))
            feature_7.append(Functions2.measure_regression(original))

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r"/Users/bogdancristianmihaila/Desktop/2nd Semester/Github/project2/Project-2-Medical-Imaging/data/images/Masks/Color_mask/107images"

# Extract features from the images
feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features(folder_path_in)

# Create a DataFrame for the features
features_df = pd.DataFrame()
features_df["img_id"] = [filename for filename in os.listdir(folder_path_in) if filename.endswith(('.jpg', '.png'))]
features_df["1: pigment network"] = feature_1
features_df["2: Blue veil"] = feature_2
features_df["3: Vascular"] = feature_3
features_df["4: Globules"] = feature_4
features_df["5: Streaks"] = feature_5
features_df["6: Pigmentation"] = feature_6
features_df["7: Regression"] = feature_7

# Merge the features DataFrame with the diagnostic column from the original DataFrame
df_merged2 = pd.merge(df[['img_id', 'diagnostic']], features_df, on='img_id', how='inner')



In [230]:
df_merged = df_merged2

In [247]:

data_path = r"/Users/bogdancristianmihaila/Desktop/2nd Semester/Github/project2/Project-2-Medical-Imaging/data/helpme/newdata.csv"
df = pd.read_csv(data_path)

new_df = pd.DataFrame()
new_df['img_id'] = df['image']
new_df['diagnostic'] = df[['MEL', 'BCC', 'AKIEC']].any(axis=1).astype(int)
df = new_df

In [248]:
# Define the function to extract features
def extract_features(folder_path):
    feature_1 = []
    feature_2 = []
    feature_3 = []
    feature_4 = []
    feature_5 = []
    feature_6 = []
    feature_7 = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            original = io.imread(image_path)

            # Ignore the alpha channel (e.g. transparency)
            if original.shape[-1] == 4:
                original = original[..., :3]

            feature_1.append(Functions2.measure_pigment_network(original))
            feature_2.append(Functions2.measure_blue_veil(original))
            feature_3.append(Functions2.measure_vascular(original))
            feature_4.append(Functions2.measure_globules(original))
            feature_5.append(Functions2.measure_streaks(original))
            feature_6.append(Functions2.measure_irregular_pigmentation(original))
            feature_7.append(Functions2.measure_regression(original))

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

In [249]:
# Define the folder path for image processing
folder_path_in = r"/Users/bogdancristianmihaila/Desktop/2nd Semester/Github/project2/Project-2-Medical-Imaging/data/both_resized"

# Extract features from the images
feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features(folder_path_in)

In [250]:
# Create a DataFrame for the features
features_df = pd.DataFrame()
features_df["img_id"] = [filename for filename in os.listdir(folder_path_in) if filename.endswith(('.jpg', '.png'))]
features_df["1: pigment network"] = feature_1
features_df["2: Blue veil"] = feature_2
features_df["3: Vascular"] = feature_3
features_df["4: Globules"] = feature_4
features_df["5: Streaks"] = feature_5
features_df["6: Pigmentation"] = feature_6
features_df["7: Regression"] = feature_7

# Merge the features DataFrame with the diagnostic column from the original DataFrame
df_merged = pd.merge(df[['img_id', 'diagnostic']], features_df, on='img_id', how='inner')

In [251]:
features_df['img_id'] = features_df['img_id'].str.replace('.jpg', '')

df_merged = pd.merge(df[['img_id', 'diagnostic']], features_df, on='img_id', how='inner')

In [236]:
#add the 2 datasets one under the other
df_merged = pd.concat([df_merged, df_merged2], ignore_index=True)
df_merged

,img_id,diagnostic,1: pigment network,2: Blue veil,3: Vascular,4: Globules,5: Streaks,6: Pigmentation,7: Regression
0,ISIC_0024306,0,22.113037,21976,21940,128,1.273240,29.417419,0
1,ISIC_0024307,0,26.664734,26870,23634,101,1.273240,40.980530,0
2,ISIC_0024308,0,23.535156,20676,23280,213,1.273240,28.671265,0
3,ISIC_0024309,0,22.734070,22935,22089,82,2.298106,34.982300,0
4,ISIC_0024310,1,15.336609,8397,12758,207,1.273240,22.416687,0
...,...,...,...,...,...,...,...,...,...
209,PAT_1730_3236_201.png,0,4.988098,4007,74,86,1.273240,7.115173,42
210,PAT_1765_3337_365.png,0,5.062866,4753,935,85,1.273240,6.770325,162
211,PAT_1942_3918_497.png,0,2.795410,29,2657,29,1.273240,3.939819,0
212,PAT_2056_4364_112.png,0,2.391052,1846,9,31,1.273240,3.442383,0


In [252]:
# Split the data into training and testing sets
X = df_merged.drop(['img_id', 'diagnostic'], axis=1)
Y = df_merged['diagnostic']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

In [253]:
df_merged

,img_id,diagnostic,1: pigment network,2: Blue veil,3: Vascular,4: Globules,5: Streaks,6: Pigmentation,7: Regression
0,ISIC_0024306,0,22.113037,21976,21940,128,1.273240,29.417419,0
1,ISIC_0024307,0,26.664734,26870,23634,101,1.273240,40.980530,0
2,ISIC_0024308,0,23.535156,20676,23280,213,1.273240,28.671265,0
3,ISIC_0024309,0,22.734070,22935,22089,82,2.298106,34.982300,0
4,ISIC_0024310,1,15.336609,8397,12758,207,1.273240,22.416687,0
...,...,...,...,...,...,...,...,...,...
102,ISIC_0024845,0,10.717773,10399,3837,54,1.273240,16.090393,0
103,ISIC_0024913,1,12.896729,12008,3287,121,1.273240,19.290161,9
104,ISIC_0024923,1,30.032349,29198,3162,140,2.959400,46.249390,0
105,ISIC_0024925,1,13.931274,13877,1077,86,1.273240,20.932007,2534


### Training classifiers without PCA

In [254]:
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score, accuracy_score

classifiers = [LR(), KNN(), DTC()]  # Replace with your trained classifiers

# Perform cross-validation for each classifier
for classifier in classifiers:
    y_pred = cross_val_predict(classifier, X, Y, cv=5)  # Change cv value as per your requirement
    
    # Calculate evaluation metrics
    f1 = f1_score(Y, y_pred)
    precision = precision_score(Y, y_pred)
    recall = recall_score(Y, y_pred)
    tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
    specificity = tn / (tn + fp)
    
    print(f"Classifier: {type(classifier).__name__}")
    print(f"F1 score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall/Sensitivity: {recall}")
    print(f"Specificity: {specificity}")
    print("Confusion Matrix:")
    print(confusion_matrix(Y, y_pred))
    print()


Classifier: LogisticRegression
F1 score: 0.8974358974358974
Precision: 0.8641975308641975
Recall/Sensitivity: 0.9333333333333333
Specificity: 0.65625
Confusion Matrix:
[[21 11]
 [ 5 70]]

Classifier: KNeighborsClassifier
F1 score: 0.8227848101265823
Precision: 0.7831325301204819
Recall/Sensitivity: 0.8666666666666667
Specificity: 0.4375
Confusion Matrix:
[[14 18]
 [10 65]]

Classifier: DecisionTreeClassifier
F1 score: 0.7272727272727272
Precision: 0.7647058823529411
Recall/Sensitivity: 0.6933333333333334
Specificity: 0.5
Confusion Matrix:
[[16 16]
 [23 52]]



### Training classifiers with PCA

In [245]:
# import numpy as np
# from sklearn.model_selection import cross_val_predict
# from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
# from sklearn.decomposition import PCA

# classifiers = [LR(), KNN(), DTC()]  # Replace with your trained classifiers

# # Perform PCA
# pca = PCA(0.95)  # Adjust the number of components as per your requirement
# X_pca = pca.fit_transform(X)

# # Perform cross-validation for each classifier
# for classifier in classifiers:
#     y_pred = cross_val_predict(classifier, X_pca, Y, cv=5)  # Change cv value as per your requirement
    
#     # Calculate evaluation metrics
#     f1 = f1_score(Y, y_pred)
#     precision = precision_score(Y, y_pred)
#     recall = recall_score(Y, y_pred)
#     tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
#     specificity = tn / (tn + fp)
    
#     print(f"Classifier: {type(classifier).__name__}")
#     print(f"F1 score: {f1}")
#     print(f"Precision: {precision}")
#     print(f"Recall/Sensitivity: {recall}")
#     print(f"Specificity: {specificity}")
#     print("Confusion Matrix:")
#     print(confusion_matrix(Y, y_pred))
#     print()

Classifier: LogisticRegression
F1 score: 0.6072874493927125
Precision: 0.635593220338983
Recall/Sensitivity: 0.5813953488372093
Specificity: 0.49411764705882355
Confusion Matrix:
[[42 43]
 [54 75]]

Classifier: KNeighborsClassifier
F1 score: 0.654275092936803
Precision: 0.6285714285714286
Recall/Sensitivity: 0.6821705426356589
Specificity: 0.38823529411764707
Confusion Matrix:
[[33 52]
 [41 88]]

Classifier: DecisionTreeClassifier
F1 score: 0.6463878326996197
Precision: 0.6343283582089553
Recall/Sensitivity: 0.6589147286821705
Specificity: 0.4235294117647059
Confusion Matrix:
[[36 49]
 [44 85]]



***Saving the models***

In [241]:
# import pickle

# # In the path where you want to save the models
# path = r'/Users/bogdancristianmihaila/Desktop/2nd Semester/Github/project2/Project-2-Medical-Imaging/Pickle/final_pickles'

# # Save the models
# #pickle.dump(LR, open(path + r'/LR_good.pkl', 'wb'))
# pickle.dump(KNN, open(path + r'/KNN_bad.pkl', 'wb'))
# #pickle.dump(DTC, open(path + r'\DTC.pkl', 'wb'))



In [255]:
from joblib import dump
dump(LR, 'LR_good.joblib')

['LR_good.joblib']